In [2]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_images_dir = '/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/train'
image_size= 224

train_datagen = ImageDataGenerator(
    rescale=1./255,  
    validation_split=0.2, 
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15
)


# Create the training generator for training data
train_generator = train_datagen.flow_from_directory(
    train_images_dir,  # Path to the training images
    target_size=(image_size, image_size),  # Resize images to target size (set image_size beforehand)
    batch_size=32,  # Number of images to process in each batch
    class_mode='categorical',  # For multi-class classification
    subset='training',  # Set subset to 'training' for training data
    shuffle=True,  # Shuffle data after each epoch
)

# Create the validation generator
validation_generator = train_datagen.flow_from_directory(
    train_images_dir, 
    target_size=(image_size, image_size),  
    batch_size=32,  
    class_mode='categorical',  
    subset='validation', 
    shuffle=False, 

)

Found 6598 images belonging to 196 classes.
Found 1546 images belonging to 196 classes.


In [3]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models, optimizers, callbacks

def identity_block(x, filters):
    f1, f2, f3 = filters
    
    shortcut = x
    x = Conv2D(f1, (1, 1), strides=(1, 1))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(f2, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(f3, (1, 1))(x)
    x = BatchNormalization()(x)

    # Add shortcut and main path
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

def convolutional_block(x, filters, strides=(2, 2)):
    f1, f2, f3 = filters

    # Shortcut Path
    shortcut = Conv2D(f3, (1, 1), strides=strides)(x)
    shortcut = BatchNormalization()(shortcut)

    # Main Path
    x = Conv2D(f1, (1, 1), strides=strides)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(f2, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(f3, (1, 1))(x)
    x = BatchNormalization()(x)

    # Add shortcut and main path
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

def ResNet50(input_shape=(224, 224, 3), classes=196):
    inputs = Input(input_shape)

    # Initial Convolutional Layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Residual Blocks
    x = convolutional_block(x, [64, 64, 256], strides=(1, 1))
    x = identity_block(x, [64, 64, 256])
    x = identity_block(x, [64, 64, 256])

    x = convolutional_block(x, [128, 128, 512])
    x = identity_block(x, [128, 128, 512])
    x = identity_block(x, [128, 128, 512])
    x = identity_block(x, [128, 128, 512])

    x = convolutional_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])

    x = convolutional_block(x, [512, 512, 2048])
    x = identity_block(x, [512, 512, 2048])
    x = identity_block(x, [512, 512, 2048])

    # Final Layers
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(196, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model


model = ResNet50(input_shape=(224, 224, 3), classes=196)
model.summary()
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Model
model = ResNet50(input_shape=(224, 224, 3), classes=196)
model.summary()

# Optimizer: SGD
optimizer = SGD(learning_rate=0.01, momentum=0.9)  # SGD with momentum
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',  # Use for one-hot encoded labels
    metrics=['accuracy']
)


reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',     # Monitor validation loss
    factor=0.002,             # Reduce learning rate by a factor of 0.5
    patience=3,             # Number of epochs with no improvement
    min_lr=1e-6,            # Minimum learning rate
    verbose=1               # Print update messages
)


# Train the model with callbacks
history = model.fit(
    train_generator,
    epochs=120,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=validation_generator.samples // 32,
    #callbacks=[reduce_lr, early_stopping],  # Include callbacks
    verbose=1  # Show detailed progress during training
)


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_106 (Conv2D) │ (None, 112, 112,  │      9,472 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_106[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_98       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 56, 56,    │          0 │ activation_98[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_108 (Conv2D) │ (None, 56, 56,    │      4,160 │ max_pooling2d_2[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_108[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_99       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_109 (Conv2D) │ (None, 56, 56,    │     36,928 │ activation_99[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_109[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_100      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_110 (Conv2D) │ (None, 56, 56,    │     16,640 │ activation_100[0… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_107 (Conv2D) │ (None, 56, 56,    │     16,640 │ max_pooling2d_2[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_110[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_107[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_32 (Add)        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 256)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_101      │ (None, 56, 56,    │          0 │ add_32[0][0]    

 Total params: 23,989,316 (91.51 MB)

 Trainable params: 23,936,196 (91.31 MB)

 Non-trainable params: 53,120 (207.50 KB)

...

187/206 ━━━━━━━━━━━━━━━━━━━━ 7s 373ms/step - accuracy: 0.0059 - loss: 6.3053

I0000 00:00:1734552047.340141     345 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_fusion_48', 24 bytes spill stores, 24 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion', 56 bytes spill stores, 56 bytes spill loads



206/206 ━━━━━━━━━━━━━━━━━━━━ 120s 412ms/step - accuracy: 0.0059 - loss: 6.2634 - val_accuracy: 0.0052 - val_loss: 5.3183
Epoch 2/50
  1/206 ━━━━━━━━━━━━━━━━━━━━ 58s 286ms/step - accuracy: 0.0000e+00 - loss: 5.5691

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


206/206 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.0000e+00 - loss: 5.5691 - val_accuracy: 0.0000e+00 - val_loss: 5.8058
Epoch 3/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 70s 334ms/step - accuracy: 0.0067 - loss: 5.3039 - val_accuracy: 0.0130 - val_loss: 5.2811
Epoch 4/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step - accuracy: 0.0312 - loss: 5.3752 - val_accuracy: 0.0000e+00 - val_loss: 6.8843
Epoch 5/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 68s 327ms/step - accuracy: 0.0140 - loss: 5.2108 - val_accuracy: 0.0117 - val_loss: 5.1448
Epoch 6/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step - accuracy: 0.0312 - loss: 5.1160 - val_accuracy: 0.0000e+00 - val_loss: 5.7806
Epoch 7/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 69s 327ms/step - accuracy: 0.0199 - loss: 5.1144 - val_accuracy: 0.0156 - val_loss: 5.6415
Epoch 8/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 254us/step - accuracy: 0.0312 - loss: 5.0586 - val_accuracy: 0.0000e+00 - val_loss: 12.5947
Epoch 9/50
206/206 ━━━━━━━━━━━━━━━━━━━━ 68s 327ms/step - accuracy: 0.0277 - lo

In [5]:
test_dir = '/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test'
batch_size=32
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=False  
)

Found 8041 images belonging to 196 classes.


In [6]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')


251/251 ━━━━━━━━━━━━━━━━━━━━ 74s 296ms/step - accuracy: 0.0722 - loss: 7.9880
Test Loss: 7.8279595375061035
Test Accuracy: 0.07457669079303741


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))

# Accuracy Curve
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss Curve
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get Ground Truth Labels and Predictions
Y_true = validation_generator.classes  # Ground truth
Y_pred = model.predict(validation_generator)
Y_pred_classes = np.argmax(Y_pred, axis=1)

# Compute Confusion Matrix
cm = confusion_matrix(Y_true, Y_pred_classes)

# Sum of misclassifications per class
misclassifications = np.sum(cm, axis=1) - np.diag(cm)

# Select Top-K Classes with Most Misclassifications
K = 10
top_k_classes = np.argsort(misclassifications)[-K:]

# Create Reduced Confusion Matrix
cm_reduced = cm[np.ix_(top_k_classes, top_k_classes)]
class_labels = np.array(list(validation_generator.class_indices.keys()))[top_k_classes]

# Display Reduced Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm_reduced, display_labels=class_labels)

plt.figure(figsize=(8, 6))
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title(f"Top-{K} Confusion Matrix")
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get Ground Truth Labels and Predictions
Y_true = validation_generator.classes  # Ground truth
Y_pred = model.predict(validation_generator)
Y_pred_classes = np.argmax(Y_pred, axis=1)

# Compute Confusion Matrix
cm = confusion_matrix(Y_true, Y_pred_classes)

# Sum of misclassifications per class
misclassifications = np.sum(cm, axis=1) - np.diag(cm)

# Select Top-K Classes with Most Misclassifications
K = 10
top_k_classes = np.argsort(misclassifications)[-K:]

# Create Reduced Confusion Matrix
cm_reduced = cm[np.ix_(top_k_classes, top_k_classes)]
class_labels = np.array(list(validation_generator.class_indices.keys()))[top_k_classes]

# Display Reduced Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm_reduced, display_labels=class_labels)

plt.figure(figsize=(8, 6))
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title(f"Top-{K} Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Generate Classification Report
print("Classification Report:\n")
report = classification_report(Y_true, Y_pred_classes, target_names=validation_generator.class_indices.keys())
print(report)